Goal of this notebook is to improve previous attempts in a more structured way

In [6]:
import torch
import torch
from torch import nn

import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
from tqdm.auto import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'

/home/aaa/Desktop/deep_learning/ConvolutionalNeuralNeutworks/venv_deep_learning/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import sys
sys.path.append('../src')


from helper_functions import train_step, test_step, accuracy_fn, print_train_time

In [8]:
seed = 213
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [9]:
cinic_directory = '../data'
cinic_train = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/train',
    	transform=transforms.ToTensor()),
    batch_size=64, shuffle=True)

cinic_test = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/test',
    	transform=transforms.ToTensor()),
    batch_size=64, shuffle=True)

In [10]:
class EnhancedModel(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        
        # Block 1: First set of convolutional layers
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, 
                      out_channels=hidden_units, 
                      kernel_size=3,
                      stride=1,
                      padding=1), 
            nn.ReLU(),
            nn.BatchNorm2d(hidden_units),  # Add batch normalization
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(hidden_units),
            nn.MaxPool2d(kernel_size=2, stride=2)  # Reduce spatial dimensions
        )
        
        # Block 2: Second set of convolutional layers
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units*2, 3, padding=1),  # Increase filter size
            nn.ReLU(),
            nn.BatchNorm2d(hidden_units*2),
            nn.Conv2d(hidden_units*2, hidden_units*2, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(hidden_units*2),
            nn.MaxPool2d(2)
        )
        
        # Block 3: Third set of convolutional layers
        self.block_3 = nn.Sequential(
            nn.Conv2d(hidden_units*2, hidden_units*4, 3, padding=1),  # Increase filter size again
            nn.ReLU(),
            nn.BatchNorm2d(hidden_units*4),
            nn.Conv2d(hidden_units*4, hidden_units*4, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(hidden_units*4),
            nn.MaxPool2d(2)
        )
        
        # Global Average Pooling to reduce the number of parameters
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        
        # Fully connected layer (classifier)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),  # Add dropout to prevent overfitting
            nn.Linear(in_features=hidden_units*4, out_features=output_shape)
        )
    
    def forward(self, x: torch.Tensor):
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.block_3(x)
        x = self.global_pool(x)  # Global average pooling
        x = self.classifier(x)
        return x


In [11]:
model = EnhancedModel(input_shape=3, 
    hidden_units=10, 
    output_shape=10).to(device)



loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [12]:
from timeit import default_timer as timer
train_time_start_model_2 = timer()

# Train and test model 
epochs = 10
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    train_step(data_loader=cinic_train, 
        model=model, 
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn,
        device=device
    )
    test_step(data_loader=cinic_test,
        model=model,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn,
        device=device
    )

train_time_end_model_2 = timer()
total_train_time_model_2 = print_train_time(start=train_time_start_model_2,
                                           end=train_time_end_model_2,
                                           device=device)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0
---------
Train loss: 1.61706 | Train accuracy: 39.55%


 10%|█         | 1/10 [00:21<03:17, 21.95s/it]

Test loss: 1.51993 | Test accuracy: 44.36%

Epoch: 1
---------
Train loss: 1.35976 | Train accuracy: 49.86%


 20%|██        | 2/10 [00:43<02:51, 21.43s/it]

...

Test loss: 1.13165 | Test accuracy: 59.59%

Epoch: 8
---------
Train loss: 1.01783 | Train accuracy: 63.27%


 90%|█████████ | 9/10 [03:10<00:21, 21.13s/it]

Test loss: 1.14819 | Test accuracy: 58.69%

Epoch: 9
---------
Train loss: 0.99918 | Train accuracy: 63.97%


100%|██████████| 10/10 [03:31<00:00, 21.20s/it]

Test loss: 1.13444 | Test accuracy: 59.67%


Train time on cuda: 211.969 seconds
